<a href="https://colab.research.google.com/github/Khomkrit007/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)



In [66]:
!pip install openpyxl
import openpyxl
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler

In [2]:
# prompt: เชื่อมต่อไปยัง drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [71]:
Data_House_data = pd.read_excel("/content/drive/MyDrive/มหาลัยขอนแก่น/3-2/Data prepare & Data Mining/bsc_dpdm24_data/Data House.xlsx", engine='openpyxl')
Id_data = pd.read_excel("/content/drive/MyDrive/มหาลัยขอนแก่น/3-2/Data prepare & Data Mining/bsc_dpdm24_data/คำตอบนักประเมิน(ID).xlsx")

##เตรียมข้อมูล Data_House

In [108]:
Data_House_data

,ID,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# prompt: แปลง id	เป็น ID

# Rename the 'id' column to 'ID' in the Data_House_data DataFrame
Data_House_data = Data_House_data.rename(columns={'id': 'ID'})

In [110]:
# Select the specified columns from the Data_House dataframe
selected_columns = ['ID','renovate','number_of_rai','number_of_ngan','number_of_va',
                    'number_of_story','number_of_bedroom','number_of_bathroom',
                    'direction','furniture','usable_area','number_of_parking',
                    'sell_price', 'district', 'sub_district', 'zone']
df = Data_House_data[selected_columns]

# Display the first few rows of the new DataFrame
df

,ID,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,1,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN
1,4,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,น้ำพอง,ทรายมูล,NaN
2,16,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN
3,18,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,19,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,no,0,0,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,NaN,437400.0,บ้านฝาง,หนองบัว,NaN
1652,20462P,no,0,0,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,NaN,714600.0,หนองเรือ,โนนทอง,NaN
1653,20463P,no,0,1,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,NaN,451000.0,แวงน้อย,ก้านเหลือง,NaN
1654,20466P,no,0,1,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,NaN,910000.0,ชุมแพ,วังหินลาด,NaN


In [111]:
# 1. Outlier Removal (Example using IQR for 'usable_area')
Q1 = df['usable_area'].quantile(0.25)
Q3 = df['usable_area'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['usable_area'] >= lower_bound) & (df['usable_area'] <= upper_bound)]

In [112]:
# 2. Missing Value Imputation (Example using mean for 'usable_area')
df['usable_area'].fillna(df['usable_area'].mean(), inplace=True)
# Apply similar imputation to other numerical columns, or use more sophisticated techniques like median or KNN

# For categorical columns, use mode (most frequent value) or a strategy appropriate for that column.
for col in ['direction', 'furniture', 'district', 'sub_district', 'zone', 'renovate']: # Replace with your actual categorical columns
  df[col].fillna(df[col].mode()[0], inplace=True)


<ipython-input-112-a223d4e85985>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['usable_area'].fillna(df['usable_area'].mean(), inplace=True)
<ipython-input-112-a223d4e85985>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['usable_area'].fillna(df['usable_area'].mean(), inplace=True)
<ipython-input-112-a223d4e85985>:7: FutureWarning: 

In [113]:
# 3. Normalization (Example using MinMaxScaler for 'usable_area' and 'sell_price')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['usable_area', 'sell_price']] = scaler.fit_transform(df[['usable_area', 'sell_price']])

<ipython-input-113-85abbacd1dba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['usable_area', 'sell_price']] = scaler.fit_transform(df[['usable_area', 'sell_price']])


In [114]:
df

,ID,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,1,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,0.579952,2.0,0.009168,ชนบท,ศรีบุญเรือง,มะลิวัลย์-ร.8-สนามบิน
1,4,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,0.389021,3.0,0.019407,น้ำพอง,ทรายมูล,มะลิวัลย์-ร.8-สนามบิน
3,18,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,0.482100,2.0,0.118943,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,19,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,0.389021,2.0,0.083224,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
5,20,no,0,0,65.8,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,0.439141,1.0,0.047506,เมืองขอนแก่น,ศิลา,ศิลา-โกทา-หนองไผ่-บ้านดอน
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,no,0,0,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,0.257184,NaN,0.010296,บ้านฝาง,หนองบัว,มะลิวัลย์-ร.8-สนามบิน
1652,20462P,no,0,0,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,0.281623,NaN,0.016897,หนองเรือ,โนนทอง,มะลิวัลย์-ร.8-สนามบิน
1653,20463P,no,0,1,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,0.447494,NaN,0.010620,แวงน้อย,ก้านเหลือง,มะลิวัลย์-ร.8-สนามบิน
1654,20466P,no,0,1,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,0.491647,NaN,0.021550,ชุมแพ,วังหินลาด,มะลิวัลย์-ร.8-สนามบิน


##เตรียมข้อมูล Id_data

In [72]:
Id_data

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [120]:
# prompt: จาก Id_data เปลื่ยนชื่อ ให้คะแนนทรัพย์ เป็น score

# Rename the 'ให้คะแนนทรัพย์' column to 'score' in the Id_data DataFrame
Id_data = Id_data.rename(columns={'ให้คะแนนทรัพย์': 'score'})

In [122]:
# prompt: colum score เอาแค่ตัวเลขหน้าหน้าเท่านั้น

# Assuming 'Id_data' DataFrame is already loaded and processed as shown in your code.

# Extract the numeric part of the 'score' column
Id_data['score'] = Id_data['score'].astype(str).str.extract('(\d+)').astype(float)

# Display the updated 'Id_data' DataFrame
Id_data

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3.0
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4.0
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2.0
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3.0
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3.0
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3.0
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3.0
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3.0


In [123]:
# prompt: เอาแค่ID ผู้ประเมิน 1

# Filter the 'Id_data' DataFrame to include only rows where the assessor ID is '01'.
assessor_id_01 = Id_data[Id_data['ID ผู้ประเมิน'] == 1]

# Display the filtered DataFrame.
assessor_id_01

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3.0
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2.0
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3.0
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,4400000.0,3500000.0,"5,000-18,000",NaN,2.0
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"5,000-25,000",NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,4950000.0,4500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,3950000.0,3500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,1700000.0,1800000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1865,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30331P,บ้านเดี่ยว,มั่นใจ,5900000.0,5500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0


In [124]:
# prompt: เปลื่ยนชื่อ colum ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')

# Rename columns
assessor_id_01 = assessor_id_01.rename(columns={'ประเมินราคาซื้อ-ขาย': 'estimator', 'ให้คะแนนทรัพย์': 'score'})

# Display the DataFrame with renamed columns
assessor_id_01

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3.0
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2.0
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3.0
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,4400000.0,3500000.0,"5,000-18,000",NaN,2.0
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"5,000-25,000",NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,4950000.0,4500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,3950000.0,3500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,1700000.0,1800000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1865,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30331P,บ้านเดี่ยว,มั่นใจ,5900000.0,5500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0


In [125]:
# Handle outliers in 'estimator' and 'score' columns
for col in ['estimator']:
    # Convert the column to numeric, errors='coerce' will replace non-numeric values with NaN
    assessor_id_01[col] = pd.to_numeric(assessor_id_01[col], errors='coerce')

    Q1 = assessor_id_01[col].quantile(0.25)
    Q3 = assessor_id_01[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    assessor_id_01 = assessor_id_01[~((assessor_id_01[col] < lower_bound) | (assessor_id_01[col] > upper_bound))]

In [126]:
# Handle missing values (replace with median for numerical columns)
for col in ['estimator']:
    assessor_id_01[col] = assessor_id_01[col].fillna(assessor_id_01[col].median())


<ipython-input-126-c4eb379d6b03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assessor_id_01[col] = assessor_id_01[col].fillna(assessor_id_01[col].median())


In [127]:
# Normalize 'estimator' and 'score' using MinMaxScaler
scaler = MinMaxScaler()
assessor_id_01[['estimator',]] = scaler.fit_transform(assessor_id_01[['estimator',]])

<ipython-input-127-6b02db0d5327>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assessor_id_01[['estimator',]] = scaler.fit_transform(assessor_id_01[['estimator',]])


In [128]:
# Display the processed DataFrame
assessor_id_01

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,0.300792,"5,000-15,000",NaN,3.0
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,0.261214,"5,000-15,000",NaN,2.0
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,0.630607,"5,000-15,000",NaN,3.0
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,4400000.0,0.432718,"5,000-18,000",NaN,2.0
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,2750000.0,0.300792,"5,000-25,000",NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
1861,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30326P,บ้านเดี่ยว,มั่นใจ,7950000.0,0.656992,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,4950000.0,0.564644,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,3950000.0,0.432718,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,1700000.0,0.208443,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2.0


##เตรียมข้อมูล eleven_data

In [ ]:
import json

# เปิดไฟล์ JSON
with open("/content/drive/MyDrive/มหาลัยขอนแก่น/3-2/Data prepare & Data Mining/bsc_dpdm24_data/7-eleven.json", 'r') as f:
    data = json.load(f)

# แสดงข้อมูล
print(data)


##เตรียมข้อมูล school_data

In [ ]:
import json

# เปิดไฟล์ JSON
with open("/content/drive/MyDrive/มหาลัยขอนแก่น/3-2/Data prepare & Data Mining/bsc_dpdm24_data/school.json", 'r') as f:
    data1 = json.load(f)

# แสดงข้อมูล
print(data1)


# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

##1.zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [64]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด  เพราะอะไร

# Group data by 'zone' and calculate the mean 'sell_price' for each zone
mean_sell_price_by_zone = df.groupby('zone')['sell_price'].mean()

# Find the zone with the highest mean 'sell_price'
highest_mean_zone = mean_sell_price_by_zone.idxmax()

print(f"The zone with the highest average selling price is: {highest_mean_zone}")

#Further analysis can be done to understand why
#Example: consider other features within that zone
#print(df[df['zone']==highest_mean_zone].describe())

The zone with the highest average selling price is: ในเมือง-รื่นรมย์-เทพารักษ์


ทำเลที่ตั้งและสิ่งอำนวยความสะดวก: โซนนี้อาจตั้งอยู่ในพื้นที่ที่น่าสนใจ มีสิ่งอำนวยความสะดวกครบครัน เช่น โรงเรียน โรงพยาบาล ห้างสรรพสินค้า และการคมนาคมที่สะดวก ปัจจัยเหล่านี้มีส่วนสำคัญในการเพิ่มมูลค่าของอสังหาริมทรัพย์

ลักษณะของอสังหาริมทรัพย์: บ้านในโซนนี้อาจมีคุณสมบัติที่เป็นที่ต้องการสูง เช่น พื้นที่ใช้สอยกว้างขวาง จำนวนห้องนอนและห้องน้ำที่มากขึ้น คุณภาพการก่อสร้างที่ดีกว่า หรือสิ่งอำนวยความสะดวกที่ทันสมัย

ปัจจัยทางเศรษฐกิจ: เศรษฐกิจในท้องถิ่นของโซนนี้อาจแข็งแกร่ง มีอัตราการจ้างงานและรายได้สูง ซึ่งสามารถกระตุ้นความต้องการและราคาที่อยู่อาศัยให้สูงขึ้น

อุปสงค์และอุปทาน: อาจมีอุปทานที่อยู่อาศัยจำกัดในโซนนี้เมื่อเทียบกับความต้องการ ซึ่งยิ่งทำให้ราคาสูงขึ้น

##3.ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [130]:
# Merge the dataframes
merged_df = pd.merge(df,Id_data, on='ID', how='inner')

# Group by 'sub_district' and calculate the mean 'score'
mean_score_by_subdistrict = merged_df.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest mean score
subdistrict_highest_score = mean_score_by_subdistrict.idxmax()

print(f"The sub_district with the highest average score is: {subdistrict_highest_score}")

The sub_district with the highest average score is: ศรีบุญเรือง


ทำไมศรีบุญเรืองจึงมีคะแนนทรัพย์เฉลี่ยสูงสุด
จากการวิเคราะห์ข้อมูลที่ได้ทำไป พบว่าตำบลศรีบุญเรืองมีคะแนนทรัพย์เฉลี่ยจากนักประเมินสูงที่สุด ซึ่งอาจเกิดจากปัจจัยต่างๆ ดังนี้

1.ทำเลที่ตั้ง: ตำบลศรีบุญเรืองอาจตั้งอยู่ในทำเลที่เดินทางสะดวก ใกล้สิ่งอำนวยความสะดวกต่างๆ เช่น โรงเรียน โรงพยาบาล ห้างสรรพสินค้า หรือสถานที่สำคัญอื่นๆ ทำให้มีความน่าสนใจสำหรับผู้ซื้อ


2.สภาพแวดล้อม: อาจมีสภาพแวดล้อมที่ดี
น่าอยู่ ปลอดภัย มีพื้นที่สีเขียว หรือบรรยากาศที่เงียบสงบ เหมาะสำหรับการอยู่อาศัย


3.คุณภาพของบ้าน: บ้านในตำบลศรีบุญเรืองอาจมีคุณภาพการก่อสร้างที่ดี ใช้วัสดุที่มีคุณภาพ มีการออกแบบที่สวยงาม ทันสมัย และตอบสนองความต้องการของผู้อยู่อาศัยได้ดี

4.โครงสร้างพื้นฐาน: อาจมีโครงสร้างพื้นฐานที่ดี เช่น ถนน ไฟฟ้า ประปา ระบบระบายน้ำ ที่ได้รับการดูแลอย่างเหมาะสม ทำให้สะดวกสบายต่อการอยู่อาศัย
ความต้องการของตลาด: อาจมีความต้องการซื้อบ้านในตำบลศรีบุญเรืองสูง ทำให้ราคาบ้านและคะแนนทรัพย์สูงขึ้นตามไปด้วย

###นายคมกฤษณ์ เลาสูงเนิน 653020601-3 SIDS